<a href="https://colab.research.google.com/github/btoneil2021/basketball-sql/blob/main/BasketballProspectShortened.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
pip install chromedriver_autoinstaller
pip install selenium
pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.5 MB/s eta 0:00:00


In [2]:
import sys
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import threading
import unicodedata
import rapidfuzz

In [3]:
# I eventually want a more efficient way of grabbing soup, but for now we chill
def get_soup(url):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  driver = webdriver.Chrome(options=chrome_options)

  driver.get(url)

  driver.implicitly_wait(1)

  soup = BeautifulSoup(driver.page_source, 'html.parser')

  driver.quit()

  return soup

In [5]:
soup = get_soup('https://www.tankathon.com/mock_draft')

mock_rows = soup.find('div', {'class': 'mock-rows'})

# Test example, grabbing all names
player_names_unstripped = mock_rows.find_all('div', {'class': 'mock-row-name'})
print(player_names_unstripped)

player_names = []
for div in player_names_unstripped:
  player_names.append(div.text)

print(player_names)

[<div class="mock-row-name">Cooper Flagg</div>, <div class="mock-row-name">Dylan Harper</div>, <div class="mock-row-name">Ace Bailey</div>, <div class="mock-row-name">V.J. Edgecombe</div>, <div class="mock-row-name">Tre Johnson</div>, <div class="mock-row-name">Kon Knueppel</div>, <div class="mock-row-name">Khaman Maluach</div>, <div class="mock-row-name">Jeremiah Fears</div>, <div class="mock-row-name">Derik Queen</div>, <div class="mock-row-name">Asa Newell</div>, <div class="mock-row-name">Kasparas Jakucionis</div>, <div class="mock-row-name">Jase Richardson</div>, <div class="mock-row-name">Collin Murray-Boyles</div>, <div class="mock-row-name">Liam McNeeley</div>, <div class="mock-row-name">Egor Demin</div>, <div class="mock-row-name">Noa Essengue</div>, <div class="mock-row-name">Nolan Traore</div>, <div class="mock-row-name">Thomas Sorber</div>, <div class="mock-row-name">Joan Beringer</div>, <div class="mock-row-name">Rasheer Fleming</div>, <div class="mock-row-name">Ben Saraf<

In [7]:
# Make links out of all the names
player_links = []

for name in player_names:
  # Normalize and remove accents
  final_name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
  # Lowercase, replace non-word chars/spaces with hyphens
  final_name = re.sub(r'\W+', '-', final_name.strip().lower())
  final_name = final_name.strip('-')
  player_links.append(f'https://www.tankathon.com/players/{final_name}')

print(player_links)

['https://www.tankathon.com/players/cooper-flagg', 'https://www.tankathon.com/players/dylan-harper', 'https://www.tankathon.com/players/ace-bailey', 'https://www.tankathon.com/players/v-j-edgecombe', 'https://www.tankathon.com/players/tre-johnson', 'https://www.tankathon.com/players/kon-knueppel', 'https://www.tankathon.com/players/khaman-maluach', 'https://www.tankathon.com/players/jeremiah-fears', 'https://www.tankathon.com/players/derik-queen', 'https://www.tankathon.com/players/asa-newell', 'https://www.tankathon.com/players/kasparas-jakucionis', 'https://www.tankathon.com/players/jase-richardson', 'https://www.tankathon.com/players/collin-murray-boyles', 'https://www.tankathon.com/players/liam-mcneeley', 'https://www.tankathon.com/players/egor-demin', 'https://www.tankathon.com/players/noa-essengue', 'https://www.tankathon.com/players/nolan-traore', 'https://www.tankathon.com/players/thomas-sorber', 'https://www.tankathon.com/players/joan-beringer', 'https://www.tankathon.com/play

In [8]:
player_soup = get_soup(player_links[0])

In [37]:
# I want to try something new: Automatically scraping all data with a single function
player_info = {}
stat_rows = player_soup.find_all('div', {'class': 'stats'})
for row in stat_rows:
  try:
    stat_container = row.find_all('div', {'class': 'stat-container'})
    for stat in stat_container:
      header = stat.find('div', {'class': 'stat-label'})
      data = stat.find('div', {'class': 'stat-data'})
      player_info[header.text] = data.text
      print(f'{header.text}: {data.text}')
  except AttributeError:
    continue
  print()

G: 37
MP: 30.7
FGM-FGA: 6.5-13.4
FG%: .481
3PM-3PA: 1.4-3.6
3P%: .385
FTM-FTA: 4.8-5.8
FT%: .840
REB: 7.5
AST: 4.2
BLK: 1.4
STL: 1.4
TO: 2.1
PF: 1.9
PTS: 19.2

G: 37
MP: 36.0
FGM-FGA: 7.6-15.8
FG%: .481
3PM-3PA: 1.6-4.3
3P%: .385
FTM-FTA: 5.7-6.8
FT%: .840
REB: 8.8
AST: 4.9
BLK: 1.6
STL: 1.6
TO: 2.5
PF: 2.2
PTS: 22.5

True Shooting %TS%: .593
Effective FG%EFG%: .533
3PA Rate3PAR: .272
FTA RateFTAR: .429
Proj NBA 3P%NBA 3P%: .367
USG%: 30.9
AST/USG: 0.87
AST/TO: 2.00

PER: 30.4
OWS/40: .180
DWS/40: .116
WS/40: .296
ORTG: 126.3
DRTG: 89.5
OBPM: 9.7
DBPM: 6.7
BPM: 16.3




In [39]:
# Same thing for the top area
top_data_section = player_soup.find_all('div', {'class': 'data-section'})
for section in top_data_section:
  try:
    block = section.find_all('div', {'class': 'data-block'})
    for metric in block:
      header = metric.find('div', {'class': 'label'})
      data = metric.find('div', {'class': 'data'})
      player_info[header.text] = data.text
      print(f'{header.text}: {data.text}')
  except AttributeError:
    continue

Team: Duke
Year: Freshman
Position: SF/PF
Height: 6'9"
Weight: 205 lbs
Mock Draft: 1st |  UTA
Big Board: 1st 
Age at Draft: 18.50 yrs
Birthdate: Dec 21, 2006
Nation:  USA
Hometown: Newport, ME
High School: Montverde AcademyMontverde Academy
ESPN 100: #1 | 2024


In [58]:
# Same for game log
player_info['Game_List'] = {}
top_row_info = []
game_log = player_soup.find('div', {'class': 'game-log'})
for index, row in enumerate(game_log.find_all('tr')):
  if index == 0:
    for header in row.find_all('td'):
      top_row_info.append(header.text)

  for index2, data in enumerate(row.find_all('td')):
    if index2 == 0:
      player_info['Game_List'][data.text] = {}
    try:
      player_info['Game_List'][data.text][top_row_info[index2]] = data.text
    except KeyError:
      continue

for game in player_info['Game_List']:
  print(game)

DATE
Nov 411/4
Nov 811/8
Nov 1211/12
Nov 1611/16
Nov 2211/22
Nov 2611/26
Nov 2911/29
Dec 412/4
Dec 812/8
Dec 1012/10
Dec 1712/17
Dec 2112/21
Dec 3112/31
Jan 41/4
Jan 71/7
Jan 111/11
Jan 141/14
Jan 181/18
Jan 251/25
Jan 271/27
Feb 12/1
Feb 52/5
Feb 82/8
Feb 122/12
Feb 152/15
Feb 172/17
Feb 222/22
Feb 252/25
Mar 13/1
Mar 33/3
Mar 83/8
CONFERENCE TOURNAMENT
Mar 133/13
NCAA TOURNAMENT
Mar 213/21
Mar 233/23
Mar 273/27
Mar 293/29
Apr 54/5
